# Clustering Neighbourhoods in Toronto

In [1]:
# Libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

#### Scaping Postal codes in Toronto
The cell below will re-create the dataframe that was created in another notebook. Only the final dataset with latitudes and longitudes will be shown.

In [2]:
# Set up the url and send request
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text

# Use beautiful soup to parse the html data
soup = BeautifulSoup(html_data, 'html.parser')

# Get table
table = soup.find_all('table')

# Get all the text boxes
textboxes = soup.find_all('td')

# Initialize dictionary
toronto = {'Postal Code': [], 'Borough': [], 'Neighbourhood': []}

# Go through the textboxes, extract the needed information
for textbox in textboxes:
    pcodes = textbox.find('b') # postal codes
    bhoods = textbox.find_all('span') # boroughs and neighbourhoods information
    
    # Not all postal codes has neighbourhood information - eliminate them
    try:
        if '(' not in bhoods[0].text:
            toronto['Neighbourhood'].append('NA')
    except: IndexError
        
    # Append extracted information to dictionary
    try:
        for pcode in pcodes:
            toronto['Postal Code'].append(pcode)
        toronto['Borough'].append(bhoods[0].text.split('(')[0])
        toronto['Neighbourhood'].append(bhoods[0].text.split('(')[1][0:-1].replace(' / ', ', '))
    except: None

# Convert the dictionary into a dataframe
df_toronto = pd.DataFrame(toronto)
        
# Remove postal codes that are not yet assigned        
df_toronto = df_toronto[df_toronto.Borough != 'Not assigned'].reset_index(drop = True)

# Get latitude and longitude information using either geocoder or this url (https://cocl.us/Geospatial_data)
df_latlng = pd.read_csv('https://cocl.us/Geospatial_data')

# Merge the postal code data from wikipedia to the latitudes and longitudes data.
df_torontolatlng = pd.merge(df_toronto, df_latlng, how = 'left', on = ['Postal Code', 'Postal Code'])
df_torontolatlng = df_torontolatlng.drop("Postal Code", axis = 1)
df_torontolatlng.head()

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Explore Neighbourhoods and Cluster them
We will now explore neighbourhoods in Toronto and cluster some of them.

In [3]:
# Get coordinates for Toronto

city = 'Toronto, Ontario'

geolocator = Nominatim(user_agent = 'toronto_explorer')
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude

print('The geographical coordinates of Toronto are {} and {}'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817 and -79.3839347


In [4]:
# Create a map of Toronto

map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

# add marks to map

for lat, lng, borough, hood in zip(df_torontolatlng['Latitude'], df_torontolatlng['Longitude'],
                                  df_torontolatlng['Borough'], df_torontolatlng['Neighbourhood']):
    label = '{}, {}'.format(hood, borough)
    label = folium.Popup(label, parse_html = True)
    
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False
    ).add_to(map_toronto)
    
map_toronto

In [5]:
# @hidden_cell
# Define Foursquare credentials

CLIENT_ID = 'IABSMDZG05PKAYRJGC3K032HVOLVO0J2M33JA0BAWQ3NJG4Q' 
CLIENT_SECRET = 'F0G53IEJMI4KCM2POOMFD5KA2Z0XFBLPEO0H5GLBQN4UT3HC' 
VERSION = '20180605' 
LIMIT = 100 # A default Foursquare API limit value

print('Foursquare credentails:')
print('CLIENT_ID: {}'.format(CLIENT_ID))
print('CLIENT_SECRET: {}'.format(CLIENT_SECRET))

Foursquare credentails:
CLIENT_ID: IABSMDZG05PKAYRJGC3K032HVOLVO0J2M33JA0BAWQ3NJG4Q
CLIENT_SECRET: F0G53IEJMI4KCM2POOMFD5KA2Z0XFBLPEO0H5GLBQN4UT3HC


We will explore boroughs that contain the words 'Toronto' or 'Scarborough' or 'York'

In [6]:
# Boroughs containing Toronto or Scarborough or York 
boroughs = ['Toronto', 'Scarborough', 'York']
boroughs_string = '|'.join(boroughs)

borough_toronto = df_torontolatlng[df_torontolatlng['Borough'].str.contains(boroughs_string)].reset_index(drop = True)
borough_toronto.head()

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Scarborough,"Malvern, Rouge",43.806686,-79.194353


#### Let us explore one neighbourhood

We will explore the third neighbourhood in our dataframe

In [7]:
hood_nm = borough_toronto.loc[2, 'Neighbourhood']
hood_lat = borough_toronto.loc[2, 'Latitude']
hood_lng = borough_toronto.loc[2, 'Longitude']

print('The latitude and longitude of {} are {} and {}.'.format(hood_nm, hood_lat, hood_lng))

The latitude and longitude of Regent Park, Harbourfront are 43.6542599 and -79.3606359.


We will now explore the St. James Town neighbourhood.\
We now create the url and get for Foursquare

In [8]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    hood_lat, 
    hood_lng, 
    radius, 
    LIMIT)

In [9]:
# Send the request and examine the results

results = requests.get(url).json()
#results # uncomment to see results

#### Function to extract the category of a venue

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean and structure the json results into a pandas dataframe

In [11]:
venues = results['response']['groups'][0]['items']

nearby_venues = pd.json_normalize(venues)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Impact Kitchen,Restaurant,43.656369,-79.356980
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [12]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

45 venues were returned by Foursquare.


Now let us generalize to obtain all the venues in the Boroughs that contain 'Toronto' or 'Scarborough' or 'York'.

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    # Initialize venues list
    venues_list = []
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        # API request url
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        # Make the get request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
borough_toronto_venues = getNearbyVenues(names = borough_toronto['Neighbourhood'],
                                        latitudes = borough_toronto['Latitude'],
                                        longitudes = borough_toronto['Longitude'])

print("There are {} venues returned for all the Boroughs in Toronto, that contains Toronto or Scarborough or York.".format(
    borough_toronto_venues.shape[0]
))

There are 1998 venues returned for all the Boroughs in Toronto, that contains Toronto or Scarborough or York.


In [15]:
borough_toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [16]:
# Number of venues returned for each neighbourd

borough_toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt),4,4,4,4,4,4
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,59,59,59,59,59,59
...,...,...,...,...,...,...
Willowdale)Wes,4,4,4,4,4,4
"Willowdale, Newtonbrook",1,1,1,1,1,1
Woburn,3,3,3,3,3,3


In [17]:
print('There are {} uniques categories.'.format(len(borough_toronto_venues['Venue Category'].unique())))

There are 261 uniques categories.


### Analyze Neighbourhoods

In [18]:
# One hot encoding
toronto_onehot = pd.get_dummies(borough_toronto_venues[['Venue Category']], prefix = "", prefix_sep = "")

# add neighbourhood column to encoding
toronto_onehot['Neighbourhood'] = borough_toronto_venues['Neighbourhood']

# Move Neighbourhood column in encoding as first column
neigh_cols = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])

toronto_onehot = toronto_onehot[neigh_cols]
toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
print('The encoded dataframe has {} rows and {} columns.'.format(toronto_onehot.shape[0], toronto_onehot.shape[1]))

The encoded dataframe has 1998 rows and 262 columns.


Group venues by neighbourhood by taking the mean of the frequency of occurrence by each category.

In [20]:
toronto_grp = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

toronto_grp.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Agincourt),0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.016949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
print('\nThe grouped dataframe has {} rows.'.format(toronto_grp.shape[0]))


The grouped dataframe has 87 rows.


Get the most common 'N' venues for each neighbourhood.

In [22]:
def return_most_common_venues(row, N):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    return row_categories_sorted.index.values[0:N]

In [23]:
N = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']

for ind in np.arange(N):
    try:
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))
        
# New dataframe
hoods_venues_sorted = pd.DataFrame(columns = columns)
hoods_venues_sorted['Neighbourhood'] = toronto_grp['Neighbourhood']

for ind in np.arange(toronto_grp.shape[0]):
    hoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grp.iloc[ind, :], N)
    
hoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt),Skating Rink,Breakfast Spot,Lounge,Latin American Restaurant,Accessories Store,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
1,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Bridal Shop,Middle Eastern Restaurant,Shopping Mall,Mobile Phone Shop,Fried Chicken Joint,Frozen Yogurt Shop,Sandwich Place,Supermarket
2,Bayview Village,Japanese Restaurant,Chinese Restaurant,Bank,Café,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Molecular Gastronomy Restaurant
3,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Hobby Shop,Pub,Butcher,Boutique,Café,Liquor Store,Sushi Restaurant
4,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Farmers Market,Restaurant,Seafood Restaurant,Cheese Shop,Pharmacy,Clothing Store


In [24]:
hoods_venues_sorted.shape

(87, 11)

### Cluster Neighbourhoods

In [25]:
# Number of clusters
clusters = 5

toronto_grp_cl = toronto_grp.drop('Neighbourhood', 1)

# KMeans clustering
km = KMeans(n_clusters = clusters, random_state = 125).fit(toronto_grp_cl)

# Check cluster labels generated
km.labels_[0:N]

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1], dtype=int32)

Create a new dataframe that includes the cluster, as well as the top 'N' venues for each neighbourhood.

In [26]:
# Add cluster labels

hoods_venues_sorted.insert(0, 'Cluster Label', km.labels_)

borough_toronto_merged = borough_toronto

# Add latitudes and longitudes information

borough_toronto_merged = borough_toronto_merged.join(hoods_venues_sorted.set_index('Neighbourhood'), on = 'Neighbourhood')
borough_toronto_merged.head()

,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Accessories Store,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Metro Station
1,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,Hockey Arena,Intersection,Portuguese Restaurant,Coffee Shop,French Restaurant,Modern European Restaurant,Motel,Moroccan Restaurant,Monument / Landmark
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Farmers Market,Event Space,Spa
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Boutique,Coffee Shop,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater
4,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Accessories Store,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


In [27]:
# Some neighbourhood cluster labels are NaN. Drop them.
borough_toronto_merged = borough_toronto_merged.dropna().reset_index(drop = True)
borough_toronto_merged.head()

,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Accessories Store,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Metro Station
1,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,Hockey Arena,Intersection,Portuguese Restaurant,Coffee Shop,French Restaurant,Modern European Restaurant,Motel,Moroccan Restaurant,Monument / Landmark
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Farmers Market,Event Space,Spa
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Boutique,Coffee Shop,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater
4,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Accessories Store,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


Visualize the clusters

In [28]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lng, hood, cluster in zip(borough_toronto_merged['Latitude'], borough_toronto_merged['Longitude'], 
                                  borough_toronto_merged['Neighbourhood'], borough_toronto_merged['Cluster Label']):
    label = folium.Popup(str(hood) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = rainbow[int(cluster - 1)],
        fill = True,
        fill_color = rainbow[int(cluster - 1)],
        fill_opacity = 0.7).add_to(map_clusters)
    
map_clusters

### Examine the Clusters

#### Cluster 1

In [29]:
borough_toronto_merged.loc[borough_toronto_merged['Cluster Label'] == 0, borough_toronto_merged.columns[[1] + list(range(5, borough_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Park,Food & Drink Shop,Accessories Store,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Metro Station
17,Caledonia-Fairbanks,Park,Women's Store,Pool,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
31,The Danforth East,Park,Intersection,Convenience Store,Rental Car Location,Accessories Store,Mobile Phone Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
36,Downsview)East,Airport,Park,Snack Place,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Metro Station
47,"Willowdale, Newtonbrook",Park,Accessories Store,Movie Theater,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
59,Weston,Park,Jewelry Store,Convenience Store,Accessories Store,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Mexican Restaurant
61,York Mills West,Park,Convenience Store,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Mexican Restaurant
77,"Milliken, Agincourt North, Steeles East, L'Amo...",Park,Playground,Intersection,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant
81,Rosedale,Park,Playground,Trail,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop


#### Cluster 2

In [30]:
borough_toronto_merged.loc[borough_toronto_merged['Cluster Label'] == 1, borough_toronto_merged.columns[[1] + list(range(5, borough_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Pizza Place,Hockey Arena,Intersection,Portuguese Restaurant,Coffee Shop,French Restaurant,Modern European Restaurant,Motel,Moroccan Restaurant,Monument / Landmark
2,"Regent Park, Harbourfront",Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Farmers Market,Event Space,Spa
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Boutique,Coffee Shop,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater
5,Don Mills)Nort,Japanese Restaurant,Gym,Caribbean Restaurant,Baseball Field,Café,Middle Eastern Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target
6,"Parkview Hill, Woodbine Gardens",Pizza Place,Pharmacy,Athletics & Sports,Flea Market,Café,Gastropub,Bank,Intersection,Breakfast Spot,Gym / Fitness Center
...,...,...,...,...,...,...,...,...,...,...,...
82,Enclave of M5E,Coffee Shop,Seafood Restaurant,Italian Restaurant,Bakery,Cocktail Bar,Hotel,Restaurant,Japanese Restaurant,Beer Bar,Café
83,"St. James Town, Cabbagetown",Coffee Shop,Café,Restaurant,Park,Bakery,Pub,Italian Restaurant,Pizza Place,Chinese Restaurant,Pet Store
84,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,Asian Restaurant,Deli / Bodega,Steakhouse,Sushi Restaurant
85,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Hotel,Mediterranean Restaurant,Men's Store,Pub,Fast Food Restaurant


#### Cluster 3

In [31]:
borough_toronto_merged.loc[borough_toronto_merged['Cluster Label'] == 2, borough_toronto_merged.columns[[1] + list(range(5, borough_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Malvern, Rouge",Fast Food Restaurant,Accessories Store,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
51,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Fast Food Restaurant,Discount Store,Sandwich Place,Accessories Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


#### Cluster 4

In [32]:
borough_toronto_merged.loc[borough_toronto_merged['Cluster Label'] == 3, borough_toronto_merged.columns[[1] + list(range(5, borough_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,The Beaches,Health Food Store,Neighborhood,Trail,Pub,Mobile Phone Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop


#### Cluster 5

In [33]:
borough_toronto_merged.loc[borough_toronto_merged['Cluster Label'] == 4, borough_toronto_merged.columns[[1] + list(range(5, borough_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Downsview)Centra,Baseball Field,Food Truck,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant
52,"Humberlea, Emery",Baseball Field,Accessories Store,Movie Theater,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
